# __TP1 - Classificador Supervisionado__

O objetivo central deste trabalho prático é desenvolver um classificador supervisionado utilizando conceitos de geometria computacional para cumprir essa tarefa.

---

#### __Disciplina:__ Algoritmos II

#### __Grupo:__
Vinicius Silva Gomes - 2021421869

Mirna

Daniel

In [27]:
# Importando bibliotecas e configurações iniciais

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functools import cmp_to_key

from math import sqrt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sys

plt.rcParams['figure.figsize']  = (16, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

plt.ion()
plt.style.use('seaborn-colorblind')

In [28]:
# URL dos Datasets

WINES = "../data/csv/wine.csv"
TIC_TAC_TOE = "../data/csv/tic-tac-toe.csv"
MUSHROOM = "../data/csv/mushroom.csv"
MAGIC = "../data/csv/magic.csv"
IRIS = "../data/csv/iris.csv"
CHESS = "../data/csv/chess.csv"
CAR = "../data/csv/car.csv"
BANANA = "../data/csv/banana.csv"
LETTER = "../data/csv/letter.csv"
TITANIC = "../data/csv/titanic.csv"

In [29]:
# Manipulação dos datasets

wines_df = pd.read_csv(WINES, sep=',')
tic_tac_toe_df = pd.read_csv(TIC_TAC_TOE, sep=',')
mushroom_df = pd.read_csv(MUSHROOM, sep=',')
magic_df = pd.read_csv(MAGIC, sep=',')
iris_df = pd.read_csv(IRIS, sep=',')
chess_df = pd.read_csv(CHESS, sep=',')
car_df = pd.read_csv(CAR, sep=',')
banana_df = pd.read_csv(BANANA, sep=',')
letter_df = pd.read_csv(LETTER, sep=',')
titanic_df = pd.read_csv(TITANIC, sep=',')

In [30]:
# Definição da classe ponto

class Point:
    def __init__(self, x = None, y = None):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"({self.x}, {self.y})"

In [31]:
# Definição da AVL que auxilia na Varredura Linear

class Node:
    def __init__(self, start = None, end = None):
        self.start = start
        self.end = end

# class AVL:

In [89]:
# Métodos auxiliares

# Plota os pontos dados como parâmetro
def plot_points(points, style='ro', labels=False, closed=False): 
    if labels:
        for (i, (x, y)) in enumerate(points):
            plt.text(x, y, '  '+str(i))
    if closed:
        points = points + [points[0]]
    plt.plot([p.x for p in points], [p.y for p in points], style)
    plt.xlim([min([p.x for p in points]) - 5, max([p.x for p in points]) + 5])
    plt.ylim([min([p.y for p in points]) - 5, max([p.y for p in points]) + 5])

# Plota a envoltória convexa dada como parâmetro
def plot_convex_hull(points, point_style="bs-", line_style="bo"):
    plot_points(points, point_style)
    plot_points(hull, line_style, closed=True)
    print(len(hull), 'of', len(points), 'points on hull')

# Posição relativa do segmento pr em relação ao segmento pq
def relative_position(p, q, r):
    # Produto vetorial dos segmentos pq e pr
    pos = ((r.x - q.x) * (q.y - p.y)) - ((r.y - q.y) * (q.x - p.x))
    
    if pos == 0:
        return 0 # colinear
    elif pos > 0:
        return 1 # sentido horário
    else:
        return -1 # sentido antihorário

## Envoltória Convexa

def find_minor_point(points):
    index = -1
    min_y = sys.maxsize
    
    for idx, point in enumerate(points):
        if point.y < min_y:
            index = idx
            min_y = point.y
        elif point.y == min_y:
            if point.x < points[index].x:
                index = idx
                
    return (points[index], index)

def euclidean_distance(p1, p2):
    distance = sqrt(((p1.x - p2.x) ** 2) + ((p1.y - p2.y) ** 2))
    
    return distance
    
def polar_sort(p0, p1, p2):
    relative_pos = relative_position(p0, p1, p2)
    
    if relative_pos == 0:
        if euclidean_distance(p0, p1) < euclidean_distance(p0, p2):
            return -1
        else:
            return 1
    else:
        if relative_pos < 0:
            return -1
        else:
            return 1
    
def next_to_top(hull):
    return hull[-2]

def top(hull):
    return hull[-1]

## Varredura linear

# Verifica se um ponto está contido dentro de um segmento
def on_segment(p, q, r):
    if r.x >= min(p.x, q.x) and r.x <= max(p.x, q.x):
        if r.y >= min(p.y, q.y) and r.y <= max(p.y, q.y):
            return True
    
    return False

# Verifica se dois segmentos se intersectam
def segments_intersect(p1, p2, p3, p4):
    d1 = relative_position(p3, p4, p1)
    d2 = relative_position(p3, p4, p2)
    d3 = relative_position(p1, p2, p3)
    d4 = relative_position(p1, p2, p4)
    
    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True
    elif d1 == 0 and on_segment(p3, p4, p1):
        return True
    elif d2 == 0 and on_segment(p3, p4, p2):
        return True
    elif d3 == 0 and on_segment(p1, p2, p3):
        return True
    elif d4 == 0 and on_segment(p1, p2, p4):
        return True
    else:
        return False

In [90]:
# Métodos principais

# Calcula a envoltória convexa para o conjunto de pontos dados
def graham_scan(points):
    # Encontra p0 no conjunto de dados
    p0, idx = find_minor_point(points)
    
    # Troca o p0 de lugar elemento de índice 0 do conjunto
    points[0], points[idx] = points[idx], points[0]
    
    # Ordena os elementos restantes a partir da coordenada polar com relação ao elemento p0
    sorted_points = points.copy()
    sorted_points[1:] = sorted(points[1:], key=cmp_to_key(lambda p1, p2: polar_sort(p0, p1, p2)))
    
    # Remove os pontos que possuem o mesmo ângulo polar em relação a p0 (mantém o que está mais longe)
#     to_remove = []
    
#     for i in range(len(sorted_points) - 1):
#         pos = relative_position(sorted_points[i], sorted_points[i + 1], p0)
        
#         if pos == 0:
#             to_remove.append(i)
    
#     sorted_points = [i for j, i in enumerate(sorted_points) if j not in to_remove]
    
    if(len(sorted_points) < 3):
        print("É impossível montar uma envoltória convexa com o conjunto de pontos fornecido!")
        return
    
    hull = []
    hull.append(sorted_points[0])
    hull.append(sorted_points[1])
    hull.append(sorted_points[2])
    
    for i in range(3, len(sorted_points)):
        while ((len(hull) > 1) and (relative_position(next_to_top(hull), top(hull), sorted_points[i]) != -1)):
            hull.pop()
            
        hull.append(sorted_points[i])
    
    return hull

# Algoritmo da varredura linear para detectar interseção entre um conjunto de pontos
# def line_sweep(points):

# Algoritmo para verificar se dois conjuntos de pontos são linearmente separável

# Algoritmo para a construção do classificador
def make_classifier(class_a, class_b):
    min_distance = sys.maxsize
    points = (None, None)
    
    # Seleciona os dois pontos, um de cada conjunto, que possuem a menor distância euclidiana entre si
    for point_a in class_a:
        for point_b in class_b:
            distance = euclidean_distance(point_a, point_b)

            if distance < min_distance:
                min_distance = distance
                points = (point_a, point_b)
                
    point_a, point_b = points
    
    # Calcula o coeficiente angular da reta
    b1 = - (1 / ((point_b.y - point_a.y) / (point_b.x - point_a.x)))
    
    # Calcula o coeficiente linear da reta
    b0 = ((point_a.y - point_b.y) / 2) - b1 * ((point_a.x - point_b.x) / 2)
                    
    # Retorna o coeficiente linear e o coeficiente angular da reta do classificador
    return (b0, b1)

In [100]:
# Conjuntos

class_a = [Point(0, 0), Point(0, 2), Point(1, 1), Point(-1, 1)]
class_b = [Point(2, 2), Point(3, 4), Point(5, 1)]

In [103]:
# Envoltória Convexa

hull_a = graham_scan(class_a)
hull_b = graham_scan(class_b)

# plot_points(class_b, style="go")
# plot_convex_hull(class_b, point_style="go", line_style="gs-")

In [104]:
# plot_points(class_a, style="bo")
# plot_convex_hull(class_a, point_style="bo", line_style="bs-")

In [94]:
# Varredura Linear

In [ ]:
# Verificar se dois conjuntos de pontos são linearmente separáveis


In [45]:
# Construção do classificador

make_classifier(class_a, class_b)

(-1.0, -1.0)